In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

/home/villacorta/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/villacorta/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


['CONTENT_CATEGORY.csv',
 '.ipynb_checkpoints',
 'pageviews_complemento.csv',
 'conversiones.csv',
 'sampleSubmission.csv',
 'pageviews.csv',
 'CONTENT_CATEGORY_TOP.csv',
 'pageview_with_previous.csv',
 'device_data.csv',
 'PAGE.csv',
 'SITE_ID.csv',
 'CONTENT_CATEGORY_BOTTOM.csv']

In [2]:
def get_schedule(val):
    if val < 6:
        return 'madrugada'
    elif val < 9:
        return 'antes_del_trabajo'
    elif val < 13:
        return 'trabajo_manana'
    elif val < 16:
        return 'almuerzo'
    elif val < 19:
        return 'trabajo_tarde'
    else:
        return 'luego_del_trabajo'
    
def get_day_cut(val):
    if val < 7:
        return 'inicio_mes'
    elif val < 13:
        return '7_12'
    elif val < 18:
        return 'quincena'
    elif val < 25:
        return '18_24'
    else:
        return 'fin_de_mes'

In [3]:
col_user = 'USER_ID'
device_data = pd.read_csv(
    "{}/device_data.csv".format(subfolder), parse_dates=["FEC_EVENT"]
).sort_values(
    [col_user, "FEC_EVENT"]
)

device_data['USER_ID'] = device_data['USER_ID'].astype(int)
device_data['CONNECTION_SPEED'] = device_data['CONNECTION_SPEED'].astype(str)
device_data['IS_MOBILE_DEVICE'] = device_data['IS_MOBILE_DEVICE'].astype(str)
device_data.shape

(2871406, 4)

In [4]:
device_data.head()

,FEC_EVENT,CONNECTION_SPEED,IS_MOBILE_DEVICE,USER_ID
572,2018-01-12 08:43:12,1,1,0
474,2018-01-12 08:59:19,1,1,0
315,2018-01-13 12:45:54,1,1,0
364,2018-01-13 12:50:15,1,1,0
187,2018-01-15 07:52:33,1,1,0


In [5]:
device_data.dtypes

FEC_EVENT           datetime64[ns]
CONNECTION_SPEED            object
IS_MOBILE_DEVICE            object
USER_ID                      int64
dtype: object

In [6]:
device_data.columns

Index(['FEC_EVENT', 'CONNECTION_SPEED', 'IS_MOBILE_DEVICE', 'USER_ID'], dtype='object')

In [7]:
data = pd.read_csv(
    "{}/pageview_with_previous.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    [col_user, "FEC_EVENT"]
)

data['USER_ID'] = data['USER_ID'].astype(int)
data.shape

(22858678, 9)

In [8]:
data.head()

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,PAGE_PREVIOUS
0,2018-01-01 09:57:19,14,4,2,4,2,1,0,14_14
1,2018-01-01 09:57:48,12,4,2,4,2,1,0,12_14
2,2018-01-01 09:57:48,10,4,2,4,2,1,0,10_12
3,2018-01-01 09:57:49,11,4,2,4,2,1,0,11_10
4,2018-01-01 10:02:09,11,4,2,4,2,1,0,11_11


In [9]:
for col in data.columns:
    print(col, data[col].unique().shape)
print("/"*100)
for col in device_data.columns:
    print(col, device_data[col].unique().shape)

FEC_EVENT (12393421,)
PAGE (1733,)
CONTENT_CATEGORY (65,)
CONTENT_CATEGORY_TOP (13,)
CONTENT_CATEGORY_BOTTOM (65,)
SITE_ID (4,)
ON_SITE_SEARCH_TERM (294,)
USER_ID (11676,)
PAGE_PREVIOUS (48516,)
////////////////////////////////////////////////////////////////////////////////////////////////////
FEC_EVENT (2592597,)
CONNECTION_SPEED (4,)
IS_MOBILE_DEVICE (2,)
USER_ID (11676,)


In [10]:
display(data.dtypes)
print("/"*100)
display(device_data.dtypes)

FEC_EVENT                  datetime64[ns]
PAGE                               object
CONTENT_CATEGORY                   object
CONTENT_CATEGORY_TOP               object
CONTENT_CATEGORY_BOTTOM            object
SITE_ID                            object
ON_SITE_SEARCH_TERM                object
USER_ID                             int64
PAGE_PREVIOUS                      object
dtype: object

////////////////////////////////////////////////////////////////////////////////////////////////////


FEC_EVENT           datetime64[ns]
CONNECTION_SPEED            object
IS_MOBILE_DEVICE            object
USER_ID                      int64
dtype: object

In [11]:
"""
new_data = []
for userx in data[col_user].unique():
    data_userx = data[data[col_user] == userx].reset_index(drop=True)
    shape_data = data_userx.shape[0]
    
    if shape_data <= 3:
        print("Only {} hiatorical:  ".format(shape_data), userx)
    
    
    if shape_data > 1:
        data_userx['PAGE_PREVIOUS'] = data_userx['PAGE'].shift(1)
        
        # data_userx['PAGE_PRE_PREVIOUS'] = data_userx['PAGE'].shift(2)
        data_userx = data_userx.drop([0],axis=0)

        #data_userx['PAGE_PREVIOUS'] = data_userx[['PAGE', 'PAGE_PREVIOUS']].apply(
        #    lambda _: str(_.PAGE) + '_' + str(_.PAGE_PREVIOUS), axis=1
        #)

        #data_userx['PAGE_PRE_PREVIOUS'] = data_userx[['PAGE_PREVIOUS', 'PAGE_PRE_PREVIOUS']].apply(
        #    lambda _: str(_.PAGE_PREVIOUS) + '_' + str(_.PAGE_PRE_PREVIOUS), axis=1
        #)

        new_data.append(data_userx)
    else:
        print("user with only one historical ::: ", userx)

data = pd.concat(new_data, axis=0)
del new_data
print(data.shape)
"""


data['PAGE_PREVIOUS'] = data['PAGE_PREVIOUS'].apply(lambda _: _.split('_')[0])
data.head()

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,PAGE_PREVIOUS
0,2018-01-01 09:57:19,14,4,2,4,2,1,0,14
1,2018-01-01 09:57:48,12,4,2,4,2,1,0,12
2,2018-01-01 09:57:48,10,4,2,4,2,1,0,10
3,2018-01-01 09:57:49,11,4,2,4,2,1,0,11
4,2018-01-01 10:02:09,11,4,2,4,2,1,0,11


In [12]:
data.isnull().sum().sum(), device_data.isnull().sum().sum()

(0, 0)

In [14]:
if 'CONTENT_CATEGORY_BOTTOM' in data.columns:
    del data['CONTENT_CATEGORY_BOTTOM']

# WEEKDAY
data['weekday_view'] = data['FEC_EVENT'].dt.weekday.astype(str)
device_data['weekday_device'] = device_data['FEC_EVENT'].dt.weekday

# SCHEDULE
data['hour'] = data['FEC_EVENT'].dt.hour
device_data['hour'] = device_data['FEC_EVENT'].dt.hour
data['schedule_view'] = data['hour'].apply(get_schedule)
device_data['schedule_device'] = device_data['hour'].apply(get_schedule)

# DAYS OF MONTH
data['day'] = data['FEC_EVENT'].dt.day
device_data['day'] = device_data['FEC_EVENT'].dt.day
data['day_view'] = data['day'].apply(get_day_cut)
device_data['day_device'] = device_data['day'].apply(get_day_cut)

del data['hour']
del device_data['hour']
del data['day']
del device_data['day']

data.isnull().sum().sum(), device_data.isnull().sum().sum()

(0, 0)

In [15]:
cols = [
  'PAGE_PREVIOUS',
  'PAGE',
  'CONTENT_CATEGORY',
  'CONTENT_CATEGORY_TOP',
  'SITE_ID',
  'ON_SITE_SEARCH_TERM',
  'weekday_view',
  'schedule_view',
  'day_view'
]

cols_device =  [
  'CONNECTION_SPEED',
  'IS_MOBILE_DEVICE',
  'weekday_device',
  'schedule_device',
  'day_device'
]

cols, cols_device

(['PAGE_PREVIOUS',
  'PAGE',
  'CONTENT_CATEGORY',
  'CONTENT_CATEGORY_TOP',
  'SITE_ID',
  'ON_SITE_SEARCH_TERM',
  'weekday_view',
  'schedule_view',
  'day_view'],
 ['CONNECTION_SPEED',
  'IS_MOBILE_DEVICE',
  'weekday_device',
  'schedule_device',
  'day_device'])

In [16]:
X_test = []

for c in cols:
    print("-|->", c)
    temp = pd.crosstab(data.USER_ID, data[c]).sort_index()
    temp.columns = [c + "__" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum() if x.sum() > 0 else x, axis=1))
    del temp

print("/"*100)

for c in cols_device:
    print("-/->", c)
    temp = pd.crosstab(device_data.USER_ID, device_data[c]).sort_index()
    temp.columns = [c + "__" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
    del temp
    
X_test = pd.concat(X_test, axis=1)

X_test.shape, X_test.isnull().sum().sum()

-|-> PAGE_PREVIOUS
-|-> PAGE
-|-> CONTENT_CATEGORY
-|-> CONTENT_CATEGORY_TOP
-|-> SITE_ID
-|-> ON_SITE_SEARCH_TERM
-|-> weekday_view
-|-> schedule_view
-|-> day_view
////////////////////////////////////////////////////////////////////////////////////////////////////
-/-> CONNECTION_SPEED
-/-> IS_MOBILE_DEVICE
-/-> weekday_device
-/-> schedule_device
-/-> day_device


((11676, 3884), 0)

In [17]:
trimestre_data = data[data.FEC_EVENT.dt.month < 10]
trimestre_data_device = device_data[device_data.FEC_EVENT.dt.month < 10]

X_train = []
X_train_device = []

for c in cols:
    print("-->", c)
    temp = pd.crosstab(trimestre_data.USER_ID, trimestre_data[c]).sort_index()
    temp.columns = [c + "__" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
    del temp

for c in cols_device:
    print("-->", c)
    temp = pd.crosstab(trimestre_data_device.USER_ID, trimestre_data_device[c]).sort_index()
    temp.columns = [c + "__" + str(v) for v in temp.columns]
    X_train_device.append(temp.apply(lambda x: x / x.sum(), axis=1))
    del temp
    
X_train = pd.concat(X_train, axis=1)
X_train_device = pd.concat(X_train_device, axis=1)

X_train[col_user] = X_train.index
del X_train.index.name
X_train_device[col_user] = X_train_device.index
del X_train_device.index.name
print(X_train.shape, X_train_device.shape)

X_train = pd.merge(X_train, X_train_device, on=col_user, how='inner')

del X_train_device
del trimestre_data
del trimestre_data_device
del data

X_train.shape

--> PAGE_PREVIOUS
--> PAGE
--> CONTENT_CATEGORY
--> CONTENT_CATEGORY_TOP
--> SITE_ID
--> ON_SITE_SEARCH_TERM
--> weekday_view
--> schedule_view
--> day_view
--> CONNECTION_SPEED
--> IS_MOBILE_DEVICE
--> weekday_device
--> schedule_device
--> day_device
(11528, 3564) (11528, 25)


(11527, 3588)

In [18]:
X_train.isnull().sum().sum(), X_train.shape, X_test.shape

(0, (11527, 3588), (11676, 3884))

In [19]:
list(X_test.columns[-10:]), list(X_train.columns[-10:])

(['schedule_device__antes_del_trabajo',
  'schedule_device__luego_del_trabajo',
  'schedule_device__madrugada',
  'schedule_device__trabajo_manana',
  'schedule_device__trabajo_tarde',
  'day_device__18_24',
  'day_device__7_12',
  'day_device__fin_de_mes',
  'day_device__inicio_mes',
  'day_device__quincena'],
 ['schedule_device__antes_del_trabajo',
  'schedule_device__luego_del_trabajo',
  'schedule_device__madrugada',
  'schedule_device__trabajo_manana',
  'schedule_device__trabajo_tarde',
  'day_device__18_24',
  'day_device__7_12',
  'day_device__fin_de_mes',
  'day_device__inicio_mes',
  'day_device__quincena'])

In [20]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
print("features: ", len(features))
X_train = X_train[features]
X_test = X_test[features]

features:  3587


In [21]:
y_prev = pd.read_csv("data/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(
        set(X_train.index))
y_train.loc[list(idx)] = 1

In [22]:
X_train.shape, X_test.shape, y_train.shape

((11527, 3587), (11676, 3587), (11527,))

In [23]:
convertion = pd.read_csv("{}/conversiones.csv".format(subfolder)).drop_duplicates()[['USER_ID','mes']].sort_values(['USER_ID','mes']).reset_index(drop=True)
for col in convertion.columns:
    convertion[col] = convertion[col].astype(int)

convertion.dtypes
def get_trimestre(val):
    if val <= 3:
        return 1
    elif val <= 6:
        return 2
    elif val <= 9:
        return 3
    elif val <= 12:
        return 4
    return 0

convertion['trimestre'] = convertion['mes'].apply(get_trimestre)
for col in convertion.columns:
    print(col, convertion[col].unique().shape)

del convertion['mes']
convertion.head()


USER_ID (1086,)
mes (12,)
trimestre (4,)


,USER_ID,trimestre
0,13,3
1,22,4
2,40,1
3,57,2
4,92,1


In [24]:
col_user = 'USER_ID'
pageview = pd.read_csv("{}/pageviews.csv".format(subfolder), parse_dates=["FEC_EVENT"]).sort_values([col_user, "FEC_EVENT"])
all_user = set(list(pageview[col_user].unique()))
print(len(all_user))
del pageview

data_conv = []

for val in sorted(list(convertion['trimestre'].unique())):
    col_target = 'TARGET_{}'.format(val)
    conv_trim = convertion[convertion['trimestre'] == val].drop_duplicates().reset_index(drop=True)
    unique_users_conv = set(list(conv_trim[col_user].unique()))
    conv_trim.index = conv_trim[col_user]
    
    conv_trim[col_target] = 1
    
    del conv_trim.index.name
    del conv_trim[col_user]
    del conv_trim['trimestre']

    temp0 = pd.DataFrame(0, index=list(all_user - unique_users_conv), columns=[col_target])
    X_test_intern = pd.concat([conv_trim, temp0]).sort_index()

    print("X_test_intern post:: ", X_test_intern.shape)
    
    data_conv.append(X_test_intern.copy())
    del X_test_intern
    del conv_trim


data_conv = pd.concat(data_conv, axis=1)
del convertion

display(data_conv.head(15))

11676
X_test_intern post::  (11676, 1)
X_test_intern post::  (11676, 1)
X_test_intern post::  (11676, 1)
X_test_intern post::  (11676, 1)


,TARGET_1,TARGET_2,TARGET_3,TARGET_4
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [25]:

current_col = 'current_convertion'
previous_col = 'previous_convertion'
before_previous_col = 'before_previous_convertion'

X_train[current_col] = data_conv['TARGET_3']
X_train[previous_col] = data_conv['TARGET_2']
X_train[before_previous_col] = data_conv['TARGET_1']

X_test[current_col] = data_conv['TARGET_4']
X_test[previous_col] = data_conv['TARGET_3']
X_test[before_previous_col] = data_conv['TARGET_2']

del data_conv

In [26]:
X_train.shape, X_test.shape, y_train.shape

((11527, 3590), (11676, 3590), (11527,))

In [27]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
num_cols = [_*int(X_train.shape[1] / 12) for _ in range(1,12)]
num_cols[:-2]

[299, 598, 897, 1196, 1495, 1794, 2093, 2392, 2691]

In [28]:
index_test = X_test.index
prediction_group = []

for num_col in num_cols[:-2]:
    sb = SelectKBest(f_classif, num_col)
    
    X_train_part = sb.fit_transform(X_train, y_train)
    X_test_part = sb.transform(X_test)
    print(X_train_part.shape, X_test_part.shape, y_train.shape)
    
    prediction_group.append(
        (X_train_part, X_test_part)
    )

(11527, 299) (11676, 299) (11527,)
(11527, 598) (11676, 598) (11527,)
(11527, 897) (11676, 897) (11527,)
(11527, 1196) (11676, 1196) (11527,)
(11527, 1495) (11676, 1495) (11527,)
(11527, 1794) (11676, 1794) (11527,)
(11527, 2093) (11676, 2093) (11527,)
(11527, 2392) (11676, 2392) (11527,)
(11527, 2691) (11676, 2691) (11527,)


In [29]:
y_train[y_train == 0].shape[0], y_train[y_train == 1].shape[0] 

(11138, 389)

In [30]:
ratio_pos = np.sum(y_train == 1) / np.sum(y_train == 0)
ratio_pos

0.03492548033758305

In [31]:
for X_train_part, X_test_part in prediction_group:
    print(X_train_part.shape, y_train.shape, X_test_part.shape)

(11527, 299) (11527,) (11676, 299)
(11527, 598) (11527,) (11676, 598)
(11527, 897) (11527,) (11676, 897)
(11527, 1196) (11527,) (11676, 1196)
(11527, 1495) (11527,) (11676, 1495)
(11527, 1794) (11527,) (11676, 1794)
(11527, 2093) (11527,) (11676, 2093)
(11527, 2392) (11527,) (11676, 2392)
(11527, 2691) (11527,) (11676, 2691)


In [32]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [35]:


#   QUIT  'scale_pos_weight': [ratio_pos], AND SET is_unbalance = True

param_grid_lgbm = {
    'n_estimators': [250, 750],
    'feature_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.95],
    'max_depth':[3, 4, 5, 6],
    'learning_rate': [0.05, 0.1, 0,0.2],
    'boosting': ['dart', 'gbdt'],
    'min_gain_to_split': [1],
    'is_unbalance': [True],
    'metric': ['auc'],
    'tree_learner': ['serial', 'feature', 'data', 'voting'],
    'xgboost_dart_mode': [False, True],
    'skip_drop': [0.45, 0.5]
}

param_grid_xgb = {
    'n_estimators': [500, 700],
    'colsample_bytree': [0.1, 0.05, 0.15],
    'subsample': [0.95],
    'max_depth':[55],
    'learning_rate': [0.025, 0.035],
    'booster': ['dart', 'gbdt'],
    'scale_pos_weight': [ratio_pos],
    'max_delta_step': [1, 5],
    'base_score': [0.5, 0.85],
    'min_child_weight':[1]
}
    
kfold_off = StratifiedKFold(
    n_splits=10, 
    shuffle=False, 
    random_state=SEED
)
model_lgbm = LGBMClassifier(
    seed=SEED, feature_fraction_seed=SEED, drop_seed=SEED
)
model_xgb = xgb.XGBClassifier(
    seed=SEED
)

for X_train_part, X_test_part in prediction_group:
    print("="*50, X_train_part.shape, X_test_part.shape)
    
    grid = GridSearchCV(
        model_lgbm, param_grid_lgbm, cv=kfold_off, n_jobs=4, scoring='roc_auc', verbose=1
    )
    grid.fit(X_train_part, y_train, eval_metric='auc')
    
    best_params_ = grid.best_params_
    model_ = grid.best_estimator_
    score_ = grid.best_score_
    print(score_, best_params_)
    print("#"*100)

================================================== (11527, 299) (11676, 299)
Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  6.4min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 27.5min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 64.3min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 113.6min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed: 140.1min finished


0.8469586765518159 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.45, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
####################################################################################################
================================================== (11527, 598) (11676, 598)
Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  9.0min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 39.1min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 92.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 162.7min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed: 202.3min finished


0.8503630425760671 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.05, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.45, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
####################################################################################################
================================================== (11527, 897) (11676, 897)
Fitting 10 folds for each of 96 candidates, totalling 960 fits


...

0.8502420678749328 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.035, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.45, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
####################################################################################################
================================================== (11527, 2093) (11676, 2093)
Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 14.1min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 60.1min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 141.0min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 247.7min
[Parallel(n_jobs=4)]: Done 960 out of 960 | elapsed: 307.9min finished


0.8470547355900317 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.5, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
####################################################################################################
================================================== (11527, 2392) (11676, 2392)
Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
"""
0 0.8469586765518159 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.45, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
1 0.8503630425760671 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.05, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.45, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
2 0.849574357074766 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.05, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.5, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
3 0.8490755891027538 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.05, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.5, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
4 0.8477170888754343 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.5, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
5 0.8502420678749328 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.035, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.45, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
6 0.8470547355900317 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.5, 'tree_learner': 'serial', 'xgboost_dart_mode': False}

"""

In [73]:
X_train_part, X_test_part = prediction_group[8]

best_parameter = {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.5, 'tree_learner': 'serial', 'xgboost_dart_mode': False}

param_grid_final = {}
for k,v in best_parameter.items():
    param_grid_final[k] = [v]

kfold_off = StratifiedKFold(
    n_splits=10, 
    shuffle=False, 
    random_state=SEED
)
model_lgbm = LGBMClassifier(
    seed=SEED, feature_fraction_seed=SEED, drop_seed=SEED
)
grid = GridSearchCV(
    model_lgbm, param_grid_final, cv=kfold_off, 
    n_jobs=-1, scoring='roc_auc', verbose=1
)
grid.fit(X_train_part, y_train)

best_params_ = grid.best_params_
model_ = grid.best_estimator_
score_ = grid.best_score_
print(score_, best_params_)

predict_test = grid.best_estimator_.predict_proba(X_test_part)[:,1]
print(predict_test, predict_test.shape)
predict_test = pd.DataFrame(
    {'USER_ID':index_test, 'SCORE':predict_test} 
)
display(predict_test.head())

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.0min finished


0.843799742876587 {'bagging_fraction': 0.95, 'boosting': 'dart', 'feature_fraction': 0.1, 'is_unbalance': True, 'learning_rate': 0.025, 'max_depth': 55, 'metric': 'auc', 'min_gain_to_split': 1, 'n_estimators': 500, 'skip_drop': 0.5, 'tree_learner': 'serial', 'xgboost_dart_mode': False}
[0.0643391  0.05722939 0.03124529 ... 0.17221013 0.2122971  0.19528629] (11676,)


,USER_ID,SCORE
0,0,0.064339
1,1,0.057229
2,2,0.031245
3,3,0.126990
4,4,0.295542


In [74]:
predict_test.shape

(11676, 2)

In [75]:
print(predict_test[predict_test['SCORE'] < 0.01].shape)
print(predict_test[predict_test['SCORE'] > 0.5].shape)
print(predict_test[predict_test['SCORE'] > 0.99].shape)

(0, 2)
(1322, 2)
(0, 2)


In [76]:
"""
ADD SCHEDULE AND WEEKDAY

(0, 2)
(1444, 2)
(0, 2)

(0, 2)
(1359, 2)
(0, 2)

(0, 2)
(1277, 2)
(0, 2)

(0, 2)
(1299, 2) -----------> 0.82098
(0, 2)

(0, 2)
(1263, 2)
(0, 2)

ADD PREVIOUS

(0, 2)
(1836, 2) -----------> 0.81141
(0, 2)

(0, 2)
(1715, 2) -----------> 0.81687    
(0, 2)

(0, 2)
(1647, 2)
(0, 2)

DATA COMPLEMENTARIA - ADD PREVIOUS

(0, 2)
(1619, 2) -------------> 0.83848
(0, 2)

(0, 2)
(1353, 2) --------------> 0.84530
(0, 2)

(0, 2)
(1352, 2)
(0, 2)

(0, 2)
(1228, 2)
(0, 2)

//////////// MORE OPTION 
1196
(0, 2)
(1412, 2)
(0, 2)


"""

'\nADD SCHEDULE AND WEEKDAY\n\n(0, 2)\n(1444, 2)\n(0, 2)\n\n(0, 2)\n(1359, 2)\n(0, 2)\n\n(0, 2)\n(1277, 2)\n(0, 2)\n\n(0, 2)\n(1299, 2) -----------> 0.82098\n(0, 2)\n\n(0, 2)\n(1263, 2)\n(0, 2)\n\nADD PREVIOUS\n\n(0, 2)\n(1836, 2) -----------> 0.81141\n(0, 2)\n\n(0, 2)\n(1715, 2) -----------> 0.81687    \n(0, 2)\n\n(0, 2)\n(1647, 2)\n(0, 2)\n\nDATA COMPLEMENTARIA - ADD PREVIOUS\n\n(0, 2)\n(1619, 2) -------------> 0.83848\n(0, 2)\n\n(0, 2)\n(1353, 2) --------------> 0.84530\n(0, 2)\n\n(0, 2)\n(1352, 2)\n(0, 2)\n\n(0, 2)\n(1228, 2)\n(0, 2)\n\n//////////// MORE OPTION \n1196\n(0, 2)\n(1412, 2)\n(0, 2)\n\n\n'

In [77]:
predict_test.to_csv('complementary_add_previous_option_sfm9-10.csv', index=False)